In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark=SparkSession.builder\
    .appName('MySparkApp')\
    .master('local[*]')\
    .getOrCreate()
spark.version

'4.0.1'

In [4]:
data = [
    (1, 1, 20),
    (2, 1, 20),
    (1, 2, 30),
    (2, 2, 30),
    (3, 2, 40),
    (1, 3, 40),
    (3, 3, 60),
    (1, 4, 60),
    (3, 4, 70)
]

In [5]:
employee_df=spark.createDataFrame(data,['id','month','salary'])
employee_df.show()

+---+-----+------+
| id|month|salary|
+---+-----+------+
|  1|    1|    20|
|  2|    1|    20|
|  1|    2|    30|
|  2|    2|    30|
|  3|    2|    40|
|  1|    3|    40|
|  3|    3|    60|
|  1|    4|    60|
|  3|    4|    70|
+---+-----+------+



In [7]:
w=Window.partitionBy('id').orderBy('month')
employee_df=employee_df.withColumn('running_amount',sum('salary').over(w))
employee_df.show()

+---+-----+------+--------------+
| id|month|salary|running_amount|
+---+-----+------+--------------+
|  1|    1|    20|            20|
|  1|    2|    30|            50|
|  1|    3|    40|            90|
|  1|    4|    60|           150|
|  2|    1|    20|            20|
|  2|    2|    30|            50|
|  3|    2|    40|            40|
|  3|    3|    60|           100|
|  3|    4|    70|           170|
+---+-----+------+--------------+



In [8]:
w=Window.partitionBy('id').orderBy(col('running_amount').desc())
employee_df=employee_df.withColumn('rnk',rank().over(w))
employee_df.show()

+---+-----+------+--------------+---+
| id|month|salary|running_amount|rnk|
+---+-----+------+--------------+---+
|  1|    4|    60|           150|  1|
|  1|    3|    40|            90|  2|
|  1|    2|    30|            50|  3|
|  1|    1|    20|            20|  4|
|  2|    2|    30|            50|  1|
|  2|    1|    20|            20|  2|
|  3|    4|    70|           170|  1|
|  3|    3|    60|           100|  2|
|  3|    2|    40|            40|  3|
+---+-----+------+--------------+---+



In [9]:
employee_df=employee_df[(employee_df['rnk']!=1)&(employee_df['rnk']<=4)]
employee_df.show()

+---+-----+------+--------------+---+
| id|month|salary|running_amount|rnk|
+---+-----+------+--------------+---+
|  1|    3|    40|            90|  2|
|  1|    2|    30|            50|  3|
|  1|    1|    20|            20|  4|
|  2|    1|    20|            20|  2|
|  3|    3|    60|           100|  2|
|  3|    2|    40|            40|  3|
+---+-----+------+--------------+---+



In [ ]:
employee_df=employee_df[['id','month',col('running_amount').alias('salary')]]
employee_df.show()

+---+-----+------+
| id|month|salary|
+---+-----+------+
|  1|    3|    90|
|  1|    2|    50|
|  1|    1|    20|
|  2|    1|    20|
|  3|    3|   100|
|  3|    2|    40|
+---+-----+------+



25/10/11 16:18:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 122963 ms exceeds timeout 120000 ms
25/10/11 16:18:04 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/11 16:18:09 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$